# Programming Assignment: 
## Готовим LDA по рецептам

Как вы уже знаете, в тематическом моделировании делается предположение о том, что для определения тематики порядок слов в документе не важен; об этом гласит гипотеза «мешка слов». Сегодня мы будем работать с несколько нестандартной для тематического моделирования коллекцией, которую можно назвать «мешком ингредиентов», потому что на состоит из рецептов блюд разных кухонь. Тематические модели ищут слова, которые часто вместе встречаются в документах, и составляют из них темы. Мы попробуем применить эту идею к рецептам и найти кулинарные «темы». Эта коллекция хороша тем, что не требует предобработки. Кроме того, эта задача достаточно наглядно иллюстрирует принцип работы тематических моделей.

Для выполнения заданий, помимо часто используемых в курсе библиотек, потребуются модули *json* и *gensim*. Первый входит в дистрибутив Anaconda, второй можно поставить командой 

*pip install gensim*

Построение модели занимает некоторое время. На ноутбуке с процессором Intel Core i7 и тактовой частотой 2400 МГц на построение одной модели уходит менее 10 минут.

### Загрузка данных

Коллекция дана в json-формате: для каждого рецепта известны его id, кухня (cuisine) и список ингредиентов, в него входящих. Загрузить данные можно с помощью модуля json (он входит в дистрибутив Anaconda):

In [1]:
import json

In [2]:
with open("recipes.json") as f:
    recipes = json.load(f)

In [4]:
print(recipes[0])

{'id': 10259, 'cuisine': 'greek', 'ingredients': ['romaine lettuce', 'black olives', 'grape tomatoes', 'garlic', 'pepper', 'purple onion', 'seasoning', 'garbanzo beans', 'feta cheese crumbles']}


In [5]:
recipes

[{'id': 10259,
  'cuisine': 'greek',
  'ingredients': ['romaine lettuce',
   'black olives',
   'grape tomatoes',
   'garlic',
   'pepper',
   'purple onion',
   'seasoning',
   'garbanzo beans',
   'feta cheese crumbles']},
 {'id': 25693,
  'cuisine': 'southern_us',
  'ingredients': ['plain flour',
   'ground pepper',
   'salt',
   'tomatoes',
   'ground black pepper',
   'thyme',
   'eggs',
   'green tomatoes',
   'yellow corn meal',
   'milk',
   'vegetable oil']},
 {'id': 20130,
  'cuisine': 'filipino',
  'ingredients': ['eggs',
   'pepper',
   'salt',
   'mayonaise',
   'cooking oil',
   'green chilies',
   'grilled chicken breasts',
   'garlic powder',
   'yellow onion',
   'soy sauce',
   'butter',
   'chicken livers']},
 {'id': 22213,
  'cuisine': 'indian',
  'ingredients': ['water', 'vegetable oil', 'wheat', 'salt']},
 {'id': 13162,
  'cuisine': 'indian',
  'ingredients': ['black pepper',
   'shallots',
   'cornflour',
   'cayenne pepper',
   'onions',
   'garlic paste',
   'm

### Составление корпуса

In [6]:
from gensim import corpora, models
import numpy as np

C:\Users\Asus\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Наша коллекция небольшая, и целиком помещается в оперативную память. Gensim может работать с такими данными и не требует их сохранения на диск в специальном формате. Для этого коллекция должна быть представлена в виде списка списков, каждый внутренний список соответствует отдельному документу и состоит из его слов. Пример коллекции из двух документов: 

[["hello", "world"], ["programming", "in", "python"]]

Преобразуем наши данные в такой формат, а затем создадим объекты corpus и dictionary, с которыми будет работать модель.

In [111]:
texts = [recipe["ingredients"] for recipe in recipes]
dictionary = corpora.Dictionary(texts)   # составляем словарь
corpus = [dictionary.doc2bow(text) for text in texts]  # составляем корпус документов

In [10]:
print(texts[0])
print(corpus[0])
print(dictionary)

['romaine lettuce', 'black olives', 'grape tomatoes', 'garlic', 'pepper', 'purple onion', 'seasoning', 'garbanzo beans', 'feta cheese crumbles']
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]
Dictionary(6714 unique tokens: ['black olives', 'feta cheese crumbles', 'garbanzo beans', 'garlic', 'grape tomatoes']...)


У объекта dictionary есть полезная переменная dictionary.token2id, позволяющая находить соответствие между ингредиентами и их индексами.

### Обучение модели
Вам может понадобиться [документация](https://radimrehurek.com/gensim/models/ldamodel.html) LDA в gensim.

__Задание 1.__ Обучите модель LDA с 40 темами, установив количество проходов по коллекции 5 и оставив остальные параметры по умолчанию. 


Затем вызовите метод модели *show_topics*, указав количество тем 40 и количество токенов 10, и сохраните результат (топы ингредиентов в темах) в отдельную переменную. Если при вызове метода *show_topics* указать параметр *formatted=True*, то топы ингредиентов будет удобно выводить на печать, если *formatted=False*, будет удобно работать со списком программно. Выведите топы на печать, рассмотрите темы, а затем ответьте на вопрос:

Сколько раз ингредиенты "salt", "sugar", "water", "mushrooms", "chicken", "eggs" встретились среди топов-10 всех 40 тем? При ответе __не нужно__ учитывать составные ингредиенты, например, "hot water".

Передайте 6 чисел в функцию save_answers1 и загрузите сгенерированный файл в форму.

У gensim нет возможности фиксировать случайное приближение через параметры метода, но библиотека использует numpy для инициализации матриц. Поэтому, по утверждению автора библиотеки, фиксировать случайное приближение нужно командой, которая написана в следующей ячейке. __Перед строкой кода с построением модели обязательно вставляйте указанную строку фиксации random.seed.__

In [112]:
np.random.seed(76543)
# здесь код для построения модели:
ldamodel = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=40, passes=5)

In [18]:
ldamodel.show_topics(num_topics=40, num_words=10, formatted=False)

[(0,
  [('chopped onion', 0.10569154),
   ('garlic cloves', 0.08284326),
   ('salt', 0.06400621),
   ('fat free less sodium chicken broth', 0.054474697),
   ('sliced green onions', 0.047655907),
   ('cooking spray', 0.047214273),
   ('ground red pepper', 0.04456691),
   ('water', 0.044441752),
   ('spinach', 0.037799563),
   ('black pepper', 0.03055882)]),
 (1,
  [('corn kernels', 0.10128137),
   ('diced onions', 0.08614667),
   ('tortillas', 0.07212572),
   ('vegetable stock', 0.06629656),
   ('chopped fresh chives', 0.04968531),
   ('sliced black olives', 0.0493395),
   ('cream cheese, soften', 0.047882482),
   ('lard', 0.0434441),
   ('jack cheese', 0.034460742),
   ('canned black beans', 0.024289647)]),
 (2,
  [('olive oil', 0.08645431),
   ('crushed red pepper', 0.059762303),
   ('fresh parsley', 0.05601106),
   ('garlic cloves', 0.041584205),
   ('grated parmesan cheese', 0.039379667),
   ('cherry tomatoes', 0.035290517),
   ('butter', 0.035038475),
   ('salt', 0.035013787),
   (

In [16]:
top_of_topics = ldamodel.show_topics(num_topics=40, num_words=10, formatted=False)

In [25]:
top_of_topics[0][1][0][0] #top_of_topics[i][1][j][0]

'chopped onion'

In [44]:
neded_words = ["salt", "sugar", "water", "mushrooms", "chicken", "eggs"]
num_of_neded_words = np.zeros(len(neded_words), dtype=int)

In [45]:
num_of_neded_words

array([0, 0, 0, 0, 0, 0])

In [46]:
for topic in top_of_topics:
    for top_word in topic[1]:
        if(top_word[0] in neded_words):
            num_of_neded_words[neded_words.index(top_word[0])]+=1
        #print(top_word[0])

In [47]:
num_of_neded_words

array([23,  9,  8,  1,  0,  2])

In [49]:
def save_answers1(mas):
    with open("cooking_LDA_pa_task1.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in mas]))

In [50]:
save_answers1(num_of_neded_words)

### Фильтрация словаря
В топах тем гораздо чаще встречаются первые три рассмотренных ингредиента, чем последние три. При этом наличие в рецепте курицы, яиц и грибов яснее дает понять, что мы будем готовить, чем наличие соли, сахара и воды. Таким образом, даже в рецептах есть слова, часто встречающиеся в текстах и не несущие смысловой нагрузки, и поэтому их не желательно видеть в темах. Наиболее простой прием борьбы с такими фоновыми элементами — фильтрация словаря по частоте. Обычно словарь фильтруют с двух сторон: убирают очень редкие слова (в целях экономии памяти) и очень частые слова (в целях повышения интерпретируемости тем). Мы уберем только частые слова.

In [92]:
import copy
dictionary2 = copy.deepcopy(dictionary)

In [54]:
print(dictionary2)

Dictionary(6714 unique tokens: ['black olives', 'feta cheese crumbles', 'garbanzo beans', 'garlic', 'grape tomatoes']...)


__Задание 2.__ У объекта dictionary2 есть переменная *dfs* — это словарь, ключами которого являются id токена, а элементами — число раз, сколько слово встретилось во всей коллекции. Сохраните в отдельный список ингредиенты, которые встретились в коллекции больше 4000 раз. Вызовите метод словаря *filter_tokens*, подав в качестве первого аргумента полученный список популярных ингредиентов. Вычислите две величины: dict_size_before и dict_size_after — размер словаря до и после фильтрации.

Затем, используя новый словарь, создайте новый корпус документов, corpus2, по аналогии с тем, как это сделано в начале ноутбука. Вычислите две величины: corpus_size_before и corpus_size_after — суммарное количество ингредиентов в корпусе (для каждого документа вычислите число различных ингредиентов в нем и просуммируйте по всем документам) до и после фильтрации.

Передайте величины dict_size_before, dict_size_after, corpus_size_before, corpus_size_after в функцию save_answers2 и загрузите сгенерированный файл в форму.

In [62]:
dictionary2.dfs

{7: 270,
 0: 229,
 4: 228,
 3: 7380,
 5: 4438,
 6: 1896,
 8: 137,
 2: 148,
 1: 358,
 14: 154,
 12: 385,
 15: 18048,
 17: 3058,
 11: 4784,
 16: 361,
 9: 3388,
 10: 108,
 19: 341,
 13: 2263,
 18: 4385,
 26: 781,
 22: 483,
 24: 768,
 25: 5,
 23: 1442,
 28: 1184,
 27: 3296,
 20: 4847,
 21: 65,
 29: 7457,
 30: 26,
 32: 2627,
 46: 1477,
 36: 103,
 34: 1523,
 44: 7972,
 39: 282,
 41: 1394,
 35: 2036,
 45: 24,
 43: 1970,
 40: 2747,
 33: 343,
 38: 925,
 37: 40,
 42: 18,
 31: 839,
 52: 6434,
 48: 338,
 49: 1231,
 53: 1298,
 50: 678,
 51: 501,
 47: 1738,
 59: 7971,
 58: 461,
 54: 796,
 57: 1730,
 56: 1094,
 61: 73,
 62: 554,
 60: 940,
 55: 103,
 66: 22,
 67: 1,
 65: 1348,
 63: 173,
 64: 51,
 71: 56,
 75: 300,
 74: 119,
 70: 965,
 68: 424,
 72: 164,
 73: 1439,
 69: 1698,
 76: 298,
 78: 1137,
 77: 2747,
 79: 3113,
 83: 134,
 87: 62,
 81: 1706,
 86: 154,
 88: 55,
 84: 164,
 80: 1223,
 82: 578,
 85: 185,
 96: 425,
 93: 1503,
 92: 152,
 94: 341,
 99: 439,
 98: 1773,
 97: 1891,
 89: 264,
 95: 168,
 91:

In [63]:
more4000_dfs = [key for key in dictionary2.dfs if dictionary2.dfs[key]>4000]

In [64]:
more4000_dfs

[3, 5, 15, 11, 18, 20, 29, 44, 52, 59, 104, 114]

In [67]:
dict_size_before = len(dictionary2)

In [99]:
dictionary2.filter_tokens(more4000_dfs)

In [70]:
dict_size_after = len(dictionary2)

In [71]:
print(dict_size_before, dict_size_after)

6714 6702


In [93]:
corpus2 = [dictionary2.doc2bow(text) for text in texts]  # составляем корпус документов

In [94]:
corpus_size_before = 0
for i in range(len(corpus2)):
    corpus_size_before+=len(corpus2[i])

In [95]:
corpus_size_before

428249

In [100]:
corpus2_filtered = [dictionary2.doc2bow(text) for text in texts]  # составляем корпус документов

In [101]:
corpus_size_after = 0
for i in range(len(corpus2_filtered)):
    corpus_size_after+=len(corpus2_filtered[i])

In [102]:
corpus_size_after

343665

In [103]:
def save_answers2(dict_size_before, dict_size_after, corpus_size_before, corpus_size_after):
    with open("cooking_LDA_pa_task2.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [dict_size_before, dict_size_after, corpus_size_before, corpus_size_after]]))

In [104]:
save_answers2(dict_size_before, dict_size_after, corpus_size_before, corpus_size_after)

### Сравнение когерентностей
__Задание 3.__ Постройте еще одну модель по корпусу corpus2 и словарю dictionary2, остальные параметры оставьте такими же, как при первом построении модели. Сохраните новую модель в другую переменную (не перезаписывайте предыдущую модель). Не забудьте про фиксирование seed!

Затем воспользуйтесь методом *top_topics* модели, чтобы вычислить ее когерентность. Передайте в качестве аргумента соответствующий модели корпус. Метод вернет список кортежей (топ токенов, когерентность), отсортированных по убыванию последней. Вычислите среднюю по всем темам когерентность для каждой из двух моделей и передайте в функцию save_answers3. 

In [107]:
np.random.seed(76543)
# здесь код для построения модели:
ldamodel2 = models.ldamodel.LdaModel(corpus2_filtered, id2word=dictionary2, num_topics=40, passes=5)

In [110]:
ldamodel2.top_topics(corpus2_filtered)

[([(0.09805376, 'sour cream'),
   (0.06260539, 'chili powder'),
   (0.05980345, 'salsa'),
   (0.059037488, 'flour tortillas'),
   (0.05172731, 'ground cumin'),
   (0.04961991, 'black beans'),
   (0.04192349, 'corn tortillas'),
   (0.026852602, 'green onions'),
   (0.026822895, 'shredded cheddar cheese'),
   (0.025870567, 'diced tomatoes'),
   (0.022652298, 'chopped cilantro fresh'),
   (0.022008648, 'shredded Monterey Jack cheese'),
   (0.019809255, 'enchilada sauce'),
   (0.01973095, 'cooked chicken'),
   (0.019138189, 'cumin'),
   (0.017684413, 'dried oregano'),
   (0.017233409, 'green chile'),
   (0.016956927, 'chicken broth'),
   (0.016009526, 'avocado'),
   (0.015510269, 'tomatoes')],
  -2.5968612427410345),
 ([(0.095774814, 'jalapeno chilies'),
   (0.07228091, 'lime'),
   (0.061941136, 'lime juice'),
   (0.0601038, 'avocado'),
   (0.046930872, 'purple onion'),
   (0.04674713, 'cilantro'),
   (0.04278179, 'chopped cilantro'),
   (0.042351104, 'fresh cilantro'),
   (0.035289135, 't

In [115]:
ldamodel2.top_topics(corpus2_filtered)[0][1]

-2.5968612427410345

In [118]:
cogers2 = [article[1] for article in ldamodel2.top_topics(corpus2_filtered)]

In [119]:
cogers2

[-2.5968612427410345,
 -2.6650788856772643,
 -2.8265769406570342,
 -3.2062340974352206,
 -3.267753040930376,
 -3.292779148771711,
 -3.479147984042398,
 -3.8252479599475553,
 -4.377012916338342,
 -4.633107489588794,
 -4.794554721495129,
 -4.813355056786778,
 -5.479868125870132,
 -5.84687960598003,
 -6.481015990558607,
 -6.659479596492677,
 -6.706127815341549,
 -6.884038793677736,
 -7.308223264629313,
 -7.819335780949607,
 -8.255341514051645,
 -8.942503822860099,
 -9.23623090484595,
 -9.481663671208771,
 -9.65640932085276,
 -9.84356402957876,
 -10.06258912200453,
 -10.21896378884116,
 -11.099547312086,
 -11.543987691405535,
 -11.687475629959795,
 -12.400385697104019,
 -12.760488245320568,
 -14.399989944219579,
 -15.571050546101239,
 -15.743775277270819,
 -15.801137438101163,
 -16.073938948472186,
 -16.62164959509965,
 -17.50178968188845]

In [121]:
coherence2 = np.mean(cogers2)
coherence2

-8.596629015979598

In [122]:
ldamodel.top_topics(corpus)

[([(0.07745518, 'sour cream'),
   (0.047240496, 'salsa'),
   (0.046635386, 'flour tortillas'),
   (0.045364756, 'chili powder'),
   (0.041672964, 'corn tortillas'),
   (0.041212466, 'shredded cheddar cheese'),
   (0.040026966, 'black beans'),
   (0.035768732, 'cilantro'),
   (0.02957908, 'salt'),
   (0.028071914, 'ground cumin'),
   (0.02611047, 'onions'),
   (0.024104077, 'avocado'),
   (0.024066417, 'cumin'),
   (0.023999808, 'green onions'),
   (0.02199853, 'jalapeno chilies'),
   (0.019403387, 'tomatoes'),
   (0.018216114, 'olive oil'),
   (0.01749658, 'pepper'),
   (0.015763303, 'corn'),
   (0.01564789, 'enchilada sauce')],
  -2.255523789444079),
 ([(0.09196267, 'soy sauce'),
   (0.051435944, 'sesame oil'),
   (0.04339177, 'scallions'),
   (0.04236651, 'green onions'),
   (0.04100772, 'rice vinegar'),
   (0.0394943, 'sugar'),
   (0.03653211, 'corn starch'),
   (0.035275135, 'garlic'),
   (0.033564173, 'vegetable oil'),
   (0.027077844, 'fresh ginger'),
   (0.025812024, 'water'),
 

In [124]:
cogers = [article[1] for article in ldamodel.top_topics(corpus)]
coherence = np.mean(cogers)
coherence

-6.338386390828257

In [125]:
def save_answers3(coherence, coherence2):
    with open("cooking_LDA_pa_task3.txt", "w") as fout:
        fout.write(" ".join(["%3f"%el for el in [coherence, coherence2]]))

In [126]:
save_answers3(coherence, coherence2)

Считается, что когерентность хорошо соотносится с человеческими оценками интерпретируемости тем. Поэтому на больших текстовых коллекциях когерентность обычно повышается, если убрать фоновую лексику. Однако в нашем случае этого не произошло. 

### Изучение влияния гиперпараметра alpha

В этом разделе мы будем работать со второй моделью, то есть той, которая построена по сокращенному корпусу. 

Пока что мы посмотрели только на матрицу темы-слова, теперь давайте посмотрим на матрицу темы-документы. Выведите темы для нулевого (или любого другого) документа из корпуса, воспользовавшись методом *get_document_topics* второй модели:

In [140]:
print(ldamodel2.get_document_topics(corpus2_filtered[0]))
words_0 = [dictionary2[num] for num, _ in ldamodel2.get_document_topics(corpus2_filtered[0])]
words_0

[(25, 0.12812185), (31, 0.61757296), (33, 0.13867696)]


['black pepper', 'garam masala', 'ground cumin']

Также выведите содержимое переменной *.alpha* второй модели:

In [138]:
ldamodel2.alpha

array([0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025,
       0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025,
       0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025,
       0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025,
       0.025, 0.025, 0.025, 0.025], dtype=float32)

У вас должно получиться, что документ характеризуется небольшим числом тем. Попробуем поменять гиперпараметр alpha, задающий априорное распределение Дирихле для распределений тем в документах.

__Задание 4.__ Обучите третью модель: используйте сокращенный корпус (corpus2 и dictionary2) и установите параметр __alpha=1__, passes=5. Не забудьте про фиксацию seed! Выведите темы новой модели для нулевого документа; должно получиться, что распределение над множеством тем практически равномерное. Чтобы убедиться в том, что во второй модели документы описываются гораздо более разреженными распределениями, чем в третьей, посчитайте суммарное количество элементов, __превосходящих 0.01__, в матрицах темы-документы обеих моделей. Другими словами, запросите темы  модели для каждого документа с параметром *minimum_probability=0.01* и просуммируйте число элементов в получаемых массивах. Передайте две суммы (сначала для модели с alpha по умолчанию, затем для модели в alpha=1) в функцию save_answers4.

In [141]:
np.random.seed(76543)
# здесь код для построения модели:
ldamodel3 = models.ldamodel.LdaModel(corpus2_filtered, id2word=dictionary2, num_topics=40, passes=5, alpha=1)

In [149]:
ldamodel3.alpha

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1.], dtype=float32)

In [142]:
print(ldamodel3.get_document_topics(corpus2_filtered[0]))
words_1 = [dictionary2[num] for num, _ in ldamodel3.get_document_topics(corpus2_filtered[0])]
words_1

[(0, 0.021397669), (1, 0.021295449), (2, 0.021276837), (3, 0.021365918), (4, 0.021295367), (5, 0.021311188), (6, 0.02130497), (7, 0.021280425), (8, 0.021401433), (9, 0.021379547), (10, 0.021837883), (11, 0.021492528), (12, 0.021276837), (13, 0.022189472), (14, 0.021718122), (15, 0.021506278), (16, 0.021404233), (17, 0.021964423), (18, 0.021329327), (19, 0.021678487), (20, 0.024654338), (21, 0.021277266), (22, 0.021276837), (23, 0.02128486), (24, 0.021771813), (25, 0.02149457), (26, 0.021462496), (27, 0.021634072), (28, 0.021495193), (29, 0.02130315), (30, 0.042615004), (31, 0.09219323), (32, 0.02150038), (33, 0.021278715), (34, 0.021446655), (35, 0.021365916), (36, 0.02133184), (37, 0.021289436), (38, 0.021277951), (39, 0.06833993)]


['black olives',
 'feta cheese crumbles',
 'garbanzo beans',
 'grape tomatoes',
 'purple onion',
 'romaine lettuce',
 'seasoning',
 'eggs',
 'green tomatoes',
 'ground pepper',
 'milk',
 'plain flour',
 'thyme',
 'tomatoes',
 'yellow corn meal',
 'chicken livers',
 'cooking oil',
 'garlic powder',
 'green chilies',
 'grilled chicken breasts',
 'mayonaise',
 'soy sauce',
 'yellow onion',
 'wheat',
 'bay leaf',
 'black pepper',
 'boneless chicken skinless thigh',
 'cayenne pepper',
 'chili powder',
 'cornflour',
 'double cream',
 'garam masala',
 'garlic paste',
 'ground cumin',
 'lemon juice',
 'natural yogurt',
 'oil',
 'passata',
 'shallots',
 'baking powder']

In [144]:
len(ldamodel3.get_document_topics(corpus2_filtered[0]))

40

In [145]:
cnt_mod2 = 0
cnt_mod3 = 0
for doc in corpus2_filtered:
    cnt_mod2 += len(ldamodel2.get_document_topics(doc, minimum_probability=0.01))
    cnt_mod3 += len(ldamodel3.get_document_topics(doc, minimum_probability=0.01))

In [147]:
print(cnt_mod2, cnt_mod3)

203705 1590960


In [148]:
def save_answers4(count_model2, count_model3):
    with open("cooking_LDA_pa_task4.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [count_model2, count_model3]]))

In [150]:
save_answers4(cnt_mod2, cnt_mod3)

Таким образом, гиперпараметр __alpha__ влияет на разреженность распределений тем в документах. Аналогично гиперпараметр __eta__ влияет на разреженность распределений слов в темах.

### LDA как способ понижения размерности
Иногда, распределения над темами, найденные с помощью LDA, добавляют в матрицу объекты-признаки как дополнительные, семантические, признаки, и это может улучшить качество решения задачи. Для простоты давайте просто обучим классификатор рецептов на кухни на признаках, полученных из LDA, и измерим точность (accuracy).

__Задание 5.__ Используйте модель, построенную по сокращенной выборке с alpha по умолчанию (вторую модель). Составьте матрицу $\Theta = p(t|d)$ вероятностей тем в документах; вы можете использовать тот же метод get_document_topics, а также вектор правильных ответов y (в том же порядке, в котором рецепты идут в переменной recipes). Создайте объект RandomForestClassifier со 100 деревьями, с помощью функции cross_val_score вычислите среднюю accuracy по трем фолдам (перемешивать данные не нужно) и передайте в функцию save_answers5.

In [152]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [170]:
teta = np.zeros((len(corpus2_filtered), 40))
for index,bow in enumerate(corpus2_filtered):
    for topic, proba in ldamodel2.get_document_topics(bow):
        teta[index, topic] = proba

In [171]:
teta

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.22499883],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.0544557 , 0.        , 0.        , ..., 0.        , 0.05623718,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.24873011]])

In [158]:
y = [recipe["cuisine"] for recipe in recipes]

In [165]:
y

['greek',
 'southern_us',
 'filipino',
 'indian',
 'indian',
 'jamaican',
 'spanish',
 'italian',
 'mexican',
 'italian',
 'italian',
 'chinese',
 'italian',
 'mexican',
 'italian',
 'indian',
 'british',
 'italian',
 'thai',
 'vietnamese',
 'thai',
 'mexican',
 'southern_us',
 'chinese',
 'italian',
 'chinese',
 'cajun_creole',
 'italian',
 'chinese',
 'mexican',
 'italian',
 'cajun_creole',
 'mexican',
 'thai',
 'italian',
 'cajun_creole',
 'italian',
 'filipino',
 'southern_us',
 'southern_us',
 'italian',
 'brazilian',
 'mexican',
 'indian',
 'mexican',
 'chinese',
 'french',
 'southern_us',
 'southern_us',
 'southern_us',
 'japanese',
 'southern_us',
 'italian',
 'southern_us',
 'italian',
 'jamaican',
 'japanese',
 'indian',
 'italian',
 'irish',
 'thai',
 'thai',
 'indian',
 'jamaican',
 'italian',
 'thai',
 'korean',
 'french',
 'french',
 'southern_us',
 'spanish',
 'indian',
 'moroccan',
 'italian',
 'italian',
 'moroccan',
 'moroccan',
 'vietnamese',
 'japanese',
 'mexican',

In [161]:
clf = RandomForestClassifier(n_estimators=100)

In [173]:
accuraces = cross_val_score(clf, teta, y, cv=3, scoring='accuracy')

In [174]:
accuraces.mean()

0.552144617086539

In [175]:
def save_answers5(accuracy):
     with open("cooking_LDA_pa_task5.txt", "w") as fout:
        fout.write(str(accuracy))

In [176]:
save_answers5(accuraces.mean())

Для такого большого количества классов это неплохая точность. Вы можете попроовать обучать RandomForest на исходной матрице частот слов, имеющей значительно большую размерность, и увидеть, что accuracy увеличивается на 10–15%. Таким образом, LDA собрал не всю, но достаточно большую часть информации из выборки, в матрице низкого ранга.

### LDA — вероятностная модель
Матричное разложение, использующееся в LDA, интерпретируется как следующий процесс генерации документов.

Для документа $d$ длины $n_d$:
1. Из априорного распределения Дирихле с параметром alpha сгенерировать распределение над множеством тем: $\theta_d \sim Dirichlet(\alpha)$
1. Для каждого слова $w = 1, \dots, n_d$:
    1. Сгенерировать тему из дискретного распределения $t \sim \theta_{d}$
    1. Сгенерировать слово из дискретного распределения $w \sim \phi_{t}$.
    
Подробнее об этом в [Википедии](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation).

В контексте нашей задачи получается, что, используя данный генеративный процесс, можно создавать новые рецепты. Вы можете передать в функцию модель и число ингредиентов и сгенерировать рецепт :)

In [178]:
def generate_recipe(model, num_ingredients):
    theta = np.random.dirichlet(model.alpha)
    for i in range(num_ingredients):
        t = np.random.choice(np.arange(model.num_topics), p=theta)
        topic = model.show_topic(t, topn=model.num_terms)
        topic_distr = [x[1] for x in topic]
        terms = [x[0] for x in topic]
        w = np.random.choice(terms, p=topic_distr)
        print(w)

In [182]:
generate_recipe(ldamodel2, 4)

ValueError: probabilities do not sum to 1

### Интерпретация построенной модели
Вы можете рассмотреть топы ингредиентов каждой темы. Большиснтво тем сами по себе похожи на рецепты; в некоторых собираются продукты одного вида, например, свежие фрукты или разные виды сыра.

Попробуем эмпирически соотнести наши темы с национальными кухнями (cuisine). Построим матрицу $A$ размера темы $x$ кухни, ее элементы $a_{tc}$ — суммы $p(t|d)$ по всем документам $d$, которые отнесены к кухне $c$. Нормируем матрицу на частоты рецептов по разным кухням, чтобы избежать дисбаланса между кухнями. Следующая функция получает на вход объект модели, объект корпуса и исходные данные и возвращает нормированную матрицу $A$. Ее удобно визуализировать с помощью seaborn.

In [183]:
import pandas
import seaborn
from matplotlib import pyplot as plt
%matplotlib inline

In [184]:
def compute_topic_cuisine_matrix(model, corpus, recipes):
    # составляем вектор целевых признаков
    targets = list(set([recipe["cuisine"] for recipe in recipes]))
    # составляем матрицу
    tc_matrix = pandas.DataFrame(data=np.zeros((model.num_topics, len(targets))), columns=targets)
    for recipe, bow in zip(recipes, corpus):
        recipe_topic = model.get_document_topics(bow)
        for t, prob in recipe_topic:
            tc_matrix[recipe["cuisine"]][t] += prob
    # нормируем матрицу
    target_sums = pandas.DataFrame(data=np.zeros((1, len(targets))), columns=targets)
    for recipe in recipes:
        target_sums[recipe["cuisine"]] += 1
    return pandas.DataFrame(tc_matrix.values/target_sums.values, columns=tc_matrix.columns)

In [185]:
def plot_matrix(tc_matrix):
    plt.figure(figsize=(10, 10))
    seaborn.heatmap(tc_matrix, square=True)

In [187]:
# Визуализируйте матрицу
matr = compute_topic_cuisine_matrix(ldamodel2, corpus2_filtered, recipes)

In [189]:
matr

,irish,thai,french,japanese,chinese,cajun_creole,spanish,filipino,southern_us,vietnamese,russian,indian,korean,moroccan,brazilian,british,mexican,italian,greek,jamaican
0,0.006582,0.018964,0.006184,0.005433,0.009612,0.013041,0.013142,0.019482,0.010955,0.016647,0.019245,0.006439,0.008367,0.005267,0.006864,0.008431,0.043138,0.016409,0.018266,0.010571
1,0.017009,0.008190,0.050671,0.009829,0.009418,0.010521,0.027028,0.008296,0.008681,0.007463,0.013926,0.007462,0.004281,0.016248,0.009026,0.018002,0.011743,0.024724,0.017004,0.007568
2,0.009373,0.014385,0.013103,0.013348,0.018029,0.009180,0.013968,0.014544,0.005530,0.012240,0.005820,0.013837,0.020983,0.016155,0.010458,0.009045,0.011960,0.022345,0.018788,0.019742
3,0.039387,0.005321,0.021074,0.008970,0.004717,0.021784,0.014129,0.015949,0.038453,0.005186,0.018473,0.014129,0.006811,0.021259,0.026245,0.032764,0.011760,0.013479,0.018542,0.089161
4,0.100676,0.006345,0.017573,0.012368,0.007869,0.022205,0.011754,0.016034,0.113497,0.005000,0.039160,0.008035,0.005255,0.006391,0.024860,0.060576,0.018666,0.017287,0.012558,0.027057
5,0.026574,0.006092,0.025018,0.009249,0.008868,0.010980,0.011884,0.012965,0.014383,0.010030,0.021337,0.016484,0.017158,0.008716,0.013926,0.017579,0.009133,0.012263,0.008539,0.012771
6,0.024622,0.007680,0.040543,0.009938,0.005542,0.025200,0.091375,0.009116,0.023778,0.007874,0.027648,0.012494,0.006507,0.036408,0.021370,0.023232,0.013367,0.074062,0.045398,0.011332
7,0.010736,0.013216,0.019499,0.005148,0.006867,0.024580,0.016507,0.013104,0.014511,0.005546,0.007717,0.006663,0.002467,0.005329,0.009813,0.004081,0.007616,0.124898,0.044665,0.005641
8,0.011187,0.014613,0.013426,0.008109,0.015372,0.008972,0.006698,0.005731,0.010592,0.014401,0.006022,0.004540,0.009570,0.007067,0.004583,0.011473,0.015829,0.007892,0.004967,0.007219
9,0.043505,0.008089,0.070666,0.017787,0.009473,0.020005,0.029454,0.014033,0.051074,0.014302,0.062513,0.018629,0.015333,0.020608,0.046949,0.070462,0.009996,0.029148,0.026098,0.015191


In [188]:
plot_matrix(matr)

Чем темнее квадрат в матрице, тем больше связь этой темы с данной кухней. Мы видим, что у нас есть темы, которые связаны с несколькими кухнями. Такие темы показывают набор ингредиентов, которые популярны в кухнях нескольких народов, то есть указывают на схожесть кухонь этих народов. Некоторые темы распределены по всем кухням равномерно, они показывают наборы продуктов, которые часто используются в кулинарии всех стран. 

Жаль, что в датасете нет названий рецептов, иначе темы было бы проще интерпретировать...

### Заключение
В этом задании вы построили несколько моделей LDA, посмотрели, на что влияют гиперпараметры модели и как можно использовать построенную модель. 